## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Neepawa,50.2289,-99.4664,36.00,83,100,24.94,CA,overcast clouds
1,1,Portland,45.5234,-122.6762,55.38,94,75,3.00,US,mist
2,2,Bredasdorp,-34.5322,20.0403,63.28,68,94,12.39,ZA,overcast clouds
3,3,Yulara,-25.2406,130.9889,84.04,17,0,10.36,AU,clear sky
4,4,Punta Arenas,-53.1500,-70.9167,55.51,41,40,16.11,CL,scattered clouds


In [17]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp= float(input('What is the minimum temperature you wouuld like for your trip?'))
max_temp= float(input('What is the maximum temperature you wouuld like for your trip?'))

What is the minimum temperature you wouuld like for your trip?70
What is the maximum temperature you wouuld like for your trip?90


In [18]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp']<= max_temp) & \
                                      (city_data_df['Max Temp']>=min_temp)].dropna()

In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             213
City                   213
Lat                    213
Lng                    213
Max Temp               213
Humidity               213
Cloudiness             213
Wind Speed             213
Country                213
Current Description    213
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

In [21]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Yulara,AU,84.04,clear sky,-25.2406,130.9889,
7,Kruisfontein,ZA,71.89,light rain,-34.0033,24.7314,
8,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,
9,South Daytona,US,77.09,few clouds,29.1658,-81.0045,
10,Port Blair,IN,80.24,moderate rain,11.6667,92.7500,
11,Isangel,VU,78.13,broken clouds,-19.5500,169.2667,
13,Dhidhdhoo,MV,82.33,few clouds,6.8833,73.1000,
14,Port Elizabeth,ZA,74.10,clear sky,-33.9180,25.5701,
15,Biloela,AU,71.56,clear sky,-24.4167,150.5000,
18,Sulangan,PH,81.32,overcast clouds,11.1373,123.7228,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng=row["Lng"]
    params['location']=f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels=requests.get(base_url,params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,'Hotel Name']=hotels['results'][0]['name']
    except(IndexError):
        print('Hotel not Found...skipping')
        

Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping
Hotel not Found...skipping


In [27]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Yulara,AU,84.04,clear sky,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
7,Kruisfontein,ZA,71.89,light rain,-34.0033,24.7314,Oyster Bay House Rental
8,Puerto Ayora,EC,76.96,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
9,South Daytona,US,77.09,few clouds,29.1658,-81.0045,Holiday Inn Express & Suites Oceanfront Dayton...
10,Port Blair,IN,80.24,moderate rain,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [28]:
# 8a. Create the output File (CSV)
output_data_file='WeatherPy_Vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations=hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
marker_layer=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))